<a href="https://colab.research.google.com/github/yunjoochoi/deep-learning-py/blob/main/4_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
# dezero.core.py

class Config: # 역전파 활성화 모드
  enable_backprop=True
  #  함수들
import contextlib
import numpy as np
@contextlib.contextmanager
def using_config(name, value):
  old_value=getattr(Config, name)
  setattr(Config, name, value)
  try:
    yield # 컨텍스트 블록 실행 (이 부분에서 실행 흐름이 사용자 코드로 넘어감)
  finally:
    setattr(Config, name, old_value)

def no_grad():
  return using_config('enable_backprop',False)

def as_array(x):
  if np.isscalar(x): #numpy.float64같은 스칼라타입인지 확인
    return np.array(x)
  return x

def as_variable(obj): # np인스턴스와 함꼐 입력되도 오류없이 수행
  if isinstance(obj, Variable):
    return obj
  return Variable(obj)

import weakref

class Variable:
  __array_priority__=200
  def __init__(self, data, name=None):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError("{}은 지원 불가".format(type(data)))
    self.data=data
    self.name=name # 많은 변수 처리 -> 이름 필요 -> 인스턴스 변수 추가
    self.grad=None
    self.creator=None
    self.generation=0

  def set_creator(self, func):
    self.creator=func
    self.generation=func.generation+1 # 부모 세대 함수보다 1만큼 큰 값 설정

  def backward(self, retain_grad=False, create_graph=False): # retain_grad: 필요 없는 미분값 삭제 / 보통 말단 변수 미분값만이 필요
    if self.grad is None:
      self.grad=Variable(np.ones_like(self.data)) # 고차 미분 구현 위해 Variable class로 감쌈

    funcs=[]
    seen_set=set()
    def add_func(f):
      if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        funcs.sort(key=lambda x:x.generation)

    add_func(self.creator)

    while funcs:
      f=funcs.pop()
      gys=[output().grad for output in f.outputs] #약한 참조로 바꿨으므로 ()추가하여 수정

      # 역전파를 단 한번만 수행하게 수정
      with using_config('enable_backprop', create_graph): # 역전파 비활성 모드로 역전파 처리 생략 (계산그래프 생성/입력 변수 유지 등 생략)
        gxs=f.backward(*gys) # 메인 역전파 (Var끼리의 연산으로 바꾸었기 때문에 f.backward내에서 순전파 실행됨. 이때 연결 생성되는데, creat_graph인수로 이를 막음. 고차 미분 시행시는 이 인수를True로 둔다)
        # gx*x1 등 불필요한 고차 미분시의 연결을 사전에 차단하는 방식.
        if not isinstance(gxs, tuple):
         gxs=(gxs,)
        for x, gx in zip(f.inputs, gxs):
          if x.grad is None:
            x.grad=gx
          else:
            x.grad = x.grad + gx
          if x.creator is not None:
            add_func(x.creator)

      if not retain_grad: # 말단 변수 아닐시 미분값 삭제
        for y in f.outputs:
          y().grad=None

  #Variable 클래스에 추가
  def reshape(self, *shape):
    if len(shape)==1 and isinstance(shape[0], (tuple, list)):
      shape=shape[0]
    return reshape(self, shape)

  def transpose(self, *axes):
    if len(axes) == 0:
      axes = None
    elif len(axes) == 1:
      if isinstance(axes[0], (tuple, list)) or axes[0] is None:
        axes = axes[0]
    return transpose(self, axes)

  def sum(self,axis=None,keepdims=False):
    return sum(self, axis, keepdims)

  def cleargrad(self):
    self.grad=None

  # 목표: Variable을 ndarray처럼 보이게 만드는 것
  @property # transpose 함수 추가
  def T(self): # 인스턴스 변수
    return transpose(self)

  @property # 인스턴스 변수처럼 사용할수 있게 함 x.shape() -> x.shape
  def shape(self):
    return self.data.shape

  @property
  def ndim(self):
    return self.data.ndim

  @property
  def size(self):
    return self.data.size

  @property
  def dtype(self):
    return self.data.dtype

  def __len__(self): # 특수 메서드
    return len(self.data)

  def __repr__(self): # print함수 출력값 수정하려면  __repr__재정의
    if self.data is None:
      return 'variable(None)'
    p=str(self.data).replace('\n','\n'+''*9)
    return 'variable('+p+')'

  # # 함수식 거추장-> 연산자 오버로드
  # def __mul__(self, other): #self=a전달, other=b전달
  #   return mul(self, other)

class Function:
  def __call__(self, *inputs):
    inputs=[as_variable(x) for x in inputs]
    xs=[x.data for x in inputs]
    ys=self.forward(*xs)
    if not isinstance(ys, tuple):
      ys=(ys,)
    outputs=[Variable(as_array(y)) for y in ys]

    if Config.enable_backprop: # 역전파 모드에서만 연결 연산 수행
      # 입력 변수 세대중에 가장 큰 세대 값으로 함수의 세대 설정
      self.generation=max([x.generation for x in inputs])
      for output in outputs:
        output.set_creator(self)
      self.inputs=inputs
      self.outputs=[weakref.ref(output) for output in outputs] # 약한 참조로 수정

    return outputs if len(outputs)>1 else outputs[0]

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):
    raise NotImplementedError()

class Square(Function):
  def forward(self, x):
    y=x**2
    return y
  def backward(self, gy):
    x=self.inputs[0].data
    gx=2*x*gy
    return gx

class Add(Function):
  def forward(self, x0,x1):
    self.x0_shape, self.x1_shape=x0.shape, x1.shape
    y=x0+x1 # 브로드캐스트 저절로 일어나는 부분(np 라서)
    return y

  def backward(self, gy):
    gx0, gx1= gy,gy
    if self.x0_shape!=self.x1_shape:
      gx0=sum_to(gx0, self.x0_shape)
      gx1=sum_to(gx1, self.x1_shape)
    return gx0,gx1 #상류에서 흘러오는 미분값을 그대로 흘려보내는 것이 덧셈의 역전파(미분시, gy*1이라서)

def square(x):
  return Square()(x)
def add(x0,x1):
  x1=as_array(x1) #x1가 float등일 경우 ndarray인스턴스로 변환
  return Add()(x0,x1)

class Mul(Function):
  def forward(self, x0, x1):
    y=x0*x1
    return y

  def backward(self, gy):
    x0, x1=self.inputs[0], self.inputs[1] # x0, x1은 Variable class (gy 도)
    return gy*x1, gy*x0 # Variable class의 *연산자는 이미 오버로드 되어있어 순전파 호출

def mul(x0,x1):
  x1=as_array(x1) #x1가 float등일 경우 ndarray인스턴스로 변환
  return Mul()(x0,x1)



class Neg(Function):
  def forward(self,x):
    return -x
  def backward(self, gy):
    return -gy
def neg(x):
  return Neg()(x)

class Sub(Function):
  def forward(self, x0,x1):
    return x0-x1
  def backward(self, gy):
    return gy,-gy

def sub(x0,x1):
  x1=as_array(x1)
  return Sub()(x0,x1)

def rsub(x0,x1):
  x1=as_array(x1)
  return Sub()(x1,x0)

class Div(Function):
  def forward(self, x0,x1):
    y=x0/x1
    return y
  def backward(self, gy):
    x0,x1=self.inputs[0], self.inputs[1]
    return gy*(1/x1), gy*(-x0/x1**2)

def div(x0,x1):
  x1=as_array(x1)
  return Div()(x0,x1)

def rdiv(x0,x1):
  x1=as_array(x1)
  return Div()(x1,x0)

class Pow(Function):
  def __init__(self, c):
    self.c=c
  def forward(self, x):
    y=x**self.c
    return y
  def backward(self, gy):
    x=self.inputs[0]
    return gy*(self.c*x**(self.c-1))

def pow(x,c):
  x=as_array(x)
  return Pow(c)(x)

def setup_variable():
  Variable.__add__=add
  Variable.__radd__=add
  Variable.__mul__=mul
  Variable.__rmul__=mul
  Variable.__rsub__=rsub
  Variable.__sub__=sub
  Variable.__neg__=neg
  Variable.__truediv__=div
  Variable.__rtruediv__=rdiv
  Variable.__pow__=pow

setup_variable()

# dezero/functions.py

# Sin함수 구현
class Sin(Function):
  def forward(self,x):
    y=np.sin(x)
    return y
  def backward(self, gy):
    x,=self.inputs # self.inputs[0] 대신 간단하게 사용
    gx=cos(x)*gy  # dezero의 cos
    return gx
def sin(x):
  return Sin()(x)

# Cos함수 구현
class Cos(Function):
  def forward(self, x):
    return np.cos(x)
  def backward(self, gy):
    x,=self.inputs
    return gy*-sin(x)
def cos(x):
  return Cos()(x)

# tanh 함수 구현
class Tanh(Function):
  def forward(self,x):
    y=np.tanh(x)
    return y
  def backward(self, gy):
    y=self.outputs[0]()
    return gy*(1-y**2)
def tanh(x):
  return Tanh()(x)

class Exp(Function):
  def forward(self, x):
    return np.exp(x)
  def backward(self, gy):
    x,=self.inputs
    return np.exp(x)*gy
def exp(x):
  return Exp()(x)

In [3]:
# 현재까지는 변수로 주로 스칼라만을 취급해왔지만, 머신러닝에서는 텐서(다차원 배열)을 사용
x=Variable(np.array([[1,2,3],[4,5,6]]))
c=Variable(np.array([[10,20,30],[40,50,60]]))
y= x + c
print(y)

variable([[11 22 33]
 [44 55 66]])


In [1]:
# x,c의 형상이 같아야 하지만 넘파이는 브로드캐스트 기능 지원
# 마지막 출력이 스칼라인 계산 그래프에 대한 역전파
# 기울기 형상과 데이터(순전파) 형상은 일치한다
# 이전까지 구현했던 스칼라별 연산 수행하는 함수에 텐서 입력해도 역전파 성립

In [3]:
x=np.array([[1,2,3],[4,5,6]])
y=np.reshape(x,(6,)) # 넘파이의 형상 변환 함수
print(y)

[1 2 3 4 5 6]


### reshape 구현

In [13]:
# 텐서 reshape함수는 아무 연산 수행X, 대신 모양만 바꾼다
class Reshape(Function):
  def __init__(self, shape):
    self.shape=shape

  def forward(self, x):
    self.x_shape=x.shape
    y=x.reshape(self.shape)
    return y

  def backward(self, gy):
    return reshape(gy, self.x_shape) # gy는 Variable인스턴스므로 dezero의 리셰입 써야함

def reshape(x, shape):
  if x.shape==shape:
    return as_variable(x)
  return Reshape(shape)(x)


In [14]:
x=Variable(np.array([[1,2,3],[4,5,6]]))
y=reshape(x, (6,))
y.backward(retain_grad=True) #y.grad = #variable([[1 1 1 1 1 1 ])
print(x.grad) #variable([[1 1 1] [1 1 1]])

variable([[1 1 1]
 [1 1 1]])


In [15]:
x=np.random.rand(1,2,3)
print(x)
y=x.reshape((2,3))
print(y)
y=x.reshape([2,3])
print(y)
y=x.reshape(2,3)
print(y)
# 모두 같은 동작, 이를 디제로로 구현

[[[0.72817274 0.72569157 0.16327641]
  [0.29343906 0.21681039 0.90523927]]]
[[0.72817274 0.72569157 0.16327641]
 [0.29343906 0.21681039 0.90523927]]
[[0.72817274 0.72569157 0.16327641]
 [0.29343906 0.21681039 0.90523927]]
[[0.72817274 0.72569157 0.16327641]
 [0.29343906 0.21681039 0.90523927]]


In [17]:
x=Variable(np.random.randn(1,2,3))
y=x.reshape((2,3))
y=x.reshape(2,3)
# 이제 인스턴스의 메서드 형태로도 호출 가능

### 행렬 전치

In [18]:
x=np.array([[1,2,3],[4,5,6]])
y=np.transpose(x)
print(y)

[[1 4]
 [2 5]
 [3 6]]


In [24]:
class Transpose(Function):
  def forward(self, x):
    y=np.transpose(x)
    return y

  def backward(self,gy):
    gx=transpose(gy)
    return gx

def transpose(x):
  return Transpose()(x)

x=np.random.rand(1,2,3,4)
x.transpose(1,0,3,2)

### 브로드캐스트 함수

In [49]:
# utils에서 가져옴
def utils_sum_to(x, shape):
    """Sum elements along axes to output an array of a given shape.

    Args:
        x (ndarray): Input array.
        shape:

    Returns:
        ndarray: Output array of the shape.
    """
    ndim = len(shape)
    lead = x.ndim - ndim
    lead_axis = tuple(range(lead))

    axis = tuple([i + lead for i, sx in enumerate(shape) if sx == 1])
    y = x.sum(lead_axis + axis, keepdims=True)
    if lead > 0:
        y = y.squeeze(lead_axis)
    return y


def reshape_sum_backward(gy, x_shape, axis, keepdims):
    """Reshape gradient appropriately for dezero.functions.sum's backward.

    Args:
        gy (dezero.Variable): Gradient variable from the output by backprop.
        x_shape (tuple): Shape used at sum function's forward.
        axis (None or int or tuple of ints): Axis used at sum function's
            forward.
        keepdims (bool): Keepdims used at sum function's forward.

    Returns:
        dezero.Variable: Gradient variable which is reshaped appropriately
    """
    ndim = len(x_shape)
    tupled_axis = axis
    if axis is None:
        tupled_axis = None
    elif not isinstance(axis, tuple):
        tupled_axis = (axis,)

    if not (ndim == 0 or tupled_axis is None or keepdims):
        actual_axis = [a if a >= 0 else a + ndim for a in tupled_axis]
        shape = list(gy.shape)
        for a in sorted(actual_axis):
            shape.insert(a, 1)
    else:
        shape = gy.shape

    gy = gy.reshape(shape)  # reshape
    return gy


In [34]:
x=np.array([1,2,3])
y=np.broadcast_to(x,(2,3))
print(y)

[[1 2 3]
 [1 2 3]]


In [41]:
x=np.array([[1,2,3],[4,5,6]])
y=sum_to(x,(1,3))
print(y)

y=sum_to(x,(2,1))
print(y)

[[5 7 9]]
[[ 6]
 [15]]


In [43]:
class BroadcastTo(Function):
  def __init__(self, shape):
    self.shape=shape
  def forward(self, x):
    self.x_shape=x.shape
    return np.broadcast_to(x, self.shape)
  def backward(self, gy):
    gx=sum_to(gy, self.x_shape)
    return gx

def broadcast_to(x, shape):
  if x.shape==shape:
    return as_variable(x)
  return BroadcastTo(shape)(x)

class SumTo(Function):
  def __init__(self, shape):
    self.shape=shape
  def forward(self, x):
    self.x_shape=x.shape
    y=utils_sum_to(x, self.shape)
    return y
  def backward(self, gy):
    gx=broadcast_to(gy, self.x_shape)
    return gx
def sum_to(x,shape):
  if x.shape==shape:
    return as_variable(x)
  return SumTo(shape)(x)

In [46]:
x0=Variable(np.array([1,2,3])) # 벡터 내에서의 덧셈연산 역전파는 가능하지만, Var끼리 덧셈시 브로드캐스트되는 경우는?
x1=Variable(np.array([10]))
y=x0+x1
print(y)
y.backward()
print(x0.grad, x1.grad) # x1.grad가 제대로 출력되지 않음
# Add class수정 필요

variable([11 12 13])
variable([1 1 1]) variable([1 1 1])


In [50]:
# 수정 후
x0=Variable(np.array([1,2,3]))
x1=Variable(np.array([10]))
y=x0+x1
print(y)
y.backward()
print(x0.grad, x1.grad)

variable([11 12 13])
variable([1 1 1]) variable([3])


### 합계 함수

In [45]:
class Sum(Function):
  def __init__(self, axis,keepdims):
    self.axis=axis
    self.keepdims=keepdims

  def forward(self, x):
    self.x_shape=x.shape
    y=x.sum(axis=self.axis, keepdims=self.keepdims)
    return y

  def backward(self,gy):
    gy=reshape_sum_backward(gy, self.x_shape, self.axis, self.keepdims) #넘파이문제로, 신경쓸필요없음
    gx=broadcast_to(gy, self.x_shape)
    return gx
def sum(x,axis=None,keepdims=False):
  return Sum(axis,keepdims)(x)

x=Variable(np.array([[1,2,3],[4,5,6]]))
y=sum(x, axis=0)
y.backward()
print(y)
print(x.grad)

variable([5 7 9])
variable([[1 1 1]
 [1 1 1]])


### 행렬의 곱

In [63]:
# 행렬의 곱과 행렬의 내적 - 넘파이
# 모두 1차원 배열이면 내적 계산
a=np.array([1,2,3])
b=np.array([4,5,6])
c=np.dot(a,b)
print(c)

# 2차원 배열 이상이면 행렬곱 계산
a=np.array([[1,2],[3,4]])
b=np.array([[5,6],[7,8]])
c=np.dot(a,b)
print(c)

32
[[19 22]
 [43 50]]


In [64]:
class MatMul(Function):
  def forward(self, x,W):
    y=x.dot(W)
    return y
  def backward(self, gy):
    x, W=self.inputs #Var형태로 받아야 밑에서 연산연결 생김
    gx=matmul(gy, W.T)
    gW=matmul(x.T, gy)
    return gx,gW

def matmul(x,W):
  return MatMul()(x,W)

In [65]:
x=Variable(np.random.randn(2,3))
W=Variable(np.random.randn(3,4))
y=matmul(x,W)
y.backward()

print(x.grad.shape) # = x.shape
print(W.grad.shape) # = W.shape

(2, 3)
(3, 4)


## 선형 회귀

In [66]:
# 토이 데이터셋
np.random.seed(0)
x=np.random.rand(100,1) # 0과 1 사이의 난수
y=5+2*x+np.random.rand(100,1) # 맨 마지막 항은 노이즈

W=Variable(np.zeros((1,1))) # x와 행렬곱하게 차원 맞춤
b=Variable(np.zeros(1))

def predict(x):
  y=matmul(x,W)+b
  return y

In [68]:
class MeanSquaredError(Function): #sample_mean_squared_error보다 불필요한 변수연결 없애서 효율 증가
  def forward(self, x0,x1):
    diff=x0-x1 # Function 상속받아서 이제 넘파이임
    y=(diff**2).sum()/len(diff)
    return y
  def backward(self, gy):
    x0,x1=self.inputs
    diff=x0-x1
    gx0=gy*diff*(2./len(diff))
    gx1=-gx0
    return gx0, gx1
def mean_squared_error(x0,x1):
  return MeanSquaredError()(x0,x1)

In [69]:
def sample_mean_squared_error(x0, x1):
  diff=x1-x0
  return sum(diff**2/len(diff))

lr=0.1
iters=100

for i in range(iters):
  y_pred=predict(x)

  loss=mean_squared_error(y, y_pred)

  W.cleargrad()
  b.cleargrad()
  loss.backward()

  W.data-=lr*W.grad.data
  b.data-=lr*b.grad.data
  print(W,b,loss)

variable([[2.11563939]]) variable([5.46732269]) variable(0.07901006311507543)
variable([[2.11323774]]) variable([5.46853978]) variable(0.07893608587689341)
variable([[2.1108683]]) variable([5.46974056]) variable(0.07886407946761541)
variable([[2.10853063]]) variable([5.47092523]) variable(0.07879399138235389)
variable([[2.10622431]]) variable([5.47209401]) variable(0.07872577051500503)
variable([[2.10394892]]) variable([5.47324712]) variable(0.07865936712098352)
variable([[2.10170404]]) variable([5.47438477]) variable(0.07859473278095046)
variable([[2.09948927]]) variable([5.47550715]) variable(0.0785318203655074)
variable([[2.0973042]]) variable([5.47661449]) variable(0.07847058400083115)
variable([[2.09514843]]) variable([5.47770698]) variable(0.07841097903522362)
variable([[2.09302157]]) variable([5.47878482]) variable(0.07835296200655356)
variable([[2.09092323]]) variable([5.4798482]) variable(0.07829649061056518)
variable([[2.08885304]]) variable([5.48089732]) variable(0.078241523

## 신경망

In [110]:
# 선형 변환= 아핀 변환 = x와W사이의 행렬곱에 b더한 연산
def linear_simple(x,W,b=None):
  t=matmul(x,W)
  if b is None:
    return t
  y= t+b
  t.data=None # 중간 계산 결과 t필요없으니 버림
  return y

class Linear(Function):
  def forward(self, x, W, b):
    y = x.dot(W)
    if b is not None:
        y += b
    return y

  def backward(self, gy):
    x, W, b = self.inputs
    gb = None if b.data is None else sum_to(gy, b.shape)
    # dim(self.grad)==dim(self.data)
    gx = matmul(gy, W.T)
    gW = matmul(x.T, gy)
    return gx, gW, gb


def linear(x, W, b=None):
  return Linear()(x, W, b)

In [72]:
# 비선형 데이터셋
np.random.seed(0)
x=np.random.rand(100,1)
y=np.sin(2*np.pi*x)+np.random.rand(100,1)

In [75]:
#비선형 변환=활성화 함수

# 메모리 효율 좋지 않은 버전
def sigmoid_simple(x):
  x=as_variable(x)
  y=1/(1 + exp(-x))
  return y

class Sigmoid(Function):
  def forward(self, x):
    y = 1 / (1 + np.exp(-x))
    y = np.tanh(x * 0.5) * 0.5 + 0.5  # Better implementation
    return y

  def backward(self, gy):
    y = self.outputs[0]()
    gx = gy * y * (1 - y)
    return gx

def sigmoid(x):
    return Sigmoid()(x)


### 신경망 구현

In [77]:
# 2층 신경망
np.random.seed(0)
x=np.random.rand(100,1)
y=np.sin(2*np.pi*x)+np.random.rand(100,1)

I, H, O=1,10,1
W1=Variable(0.01*np.random.randn(I, H))
b1=Variable(np.zeros(H))
W2=Variable(0.01*np.random.randn(H, O))
b2=Variable(np.zeros(O))

def predict(x):
  y=linear(x,W1,b1)
  y=sigmoid(y)
  y=linear(y,W2,b2)
  return y

lr=0.2
iters=10000

for i in range(iters):
  y_pred=predict(x)
  loss=mean_squared_error(y, y_pred)

  W1.cleargrad()
  b1.cleargrad()
  b2.cleargrad()
  W2.cleargrad()
  loss.backward()

  W1.data-=lr*W1.grad.data
  W2.data-=lr*W2.grad.data
  b1.data-=lr*b1.grad.data
  b2.data-=lr*b2.grad.data
  if i%1000==0:
    print(loss)

variable(0.8473695850105871)
variable(0.2514286285183607)
variable(0.24759485466749878)
variable(0.23786120447054832)
variable(0.21222231333102953)
variable(0.16742181117834223)
variable(0.0968193261999272)
variable(0.07849528290602335)
variable(0.07749729552991157)
variable(0.07722132399559317)


## 매개변수를 모아두는 계층

In [79]:
# 매개변수 기울기 재설정 시 단조로운 코드의 반복
# 매개변수는 경사하강법을 통해 업데이트됨
# Parameter 클래스 구축 Variable의 기능 필요->상속

class Parameter(Variable):
  pass

x=Variable(np.array(1.0))
p=Parameter(np.array(1.0))
y=x*p

print(isinstance(p, Parameter))
print(isinstance(x, Parameter))
print(isinstance(y, Parameter))
print(y)

True
False
False
variable(1.0)


In [95]:
import weakref
class Layer:
  def __init__(self):
    self._params=set() # layer 인스턴스에 속한 매개변수를 보관
  def __setattr__(self, name, value): # 인스턴스 변수 설정시 호출되는 특수 메서드
    if isinstance(value, Parameter):
      self._params.add(name)
    super().__setattr__(name,value)

  def __call__(self, *inputs):
    outputs=self.forward(*inputs)
    if not isinstance(outputs, tuple):
      outputs=(outputs,)
    self.inputs=[weakref.ref(x) for x in inputs]
    self.outputs=[weakref.ref(x) for x in outputs]
    return outputs if len(outputs)>1 else outputs[0]

  def forward(self, inputs):
    raise NotImplementedError()

  def params(self): # Layer인스턴스에 담겨있는 Parameter인스턴스들을 꺼내준다.
    for name in self._params:
      yield self.__dict__[name] #return과 비슷하게 값을 반환하지만, 함수를 종료하지 않고 실행 상태를 저장한 채 멈춥니다.다음번에 호출하면 멈췄던 지점부터 다시 실행
  # param 메서드 호출 시마다 일시 정지됬던 처리가 재개됨.

  def cleargrads(self): # 모든 매개변수 기울기를 재설정한다.
    for param in self.params():
      param.cleargrad()

layer=Layer()
layer.p1=Parameter(np.array(1))
layer.p2=Parameter(np.array(2))
layer.p3=Variable(np.array(3))
layer.p4='test'

print(layer._params)
print('------------')

for name in layer._params:
  print(name, layer.__dict__[name]) # __dict__에는 모든 인스턴스 변수가 딕셔너리 타입으로 저장됨


{'p2', 'p1'}
------------
p2 variable(2)
p1 variable(1)


In [102]:
# Linear class 구현
# class Linear(Layer):
#   def __init__(self, in_size, out_size, nobias=False, dtype=np.float32):
#     super().__init__()

#     I,O =in_size,out_size
#     W_data=np.random.randn(I,O).astype(dtype) * np.sqrt(1/I)
#     self.W=Parameter(W_data, name='W')
#     if nobias:
#       self.b=None
#     else:
#       self.b=Parameter(np.zeros(O, dtype=dtype),name='b')

#   def forward(self, x):
#     y=linear(x, self.W, self.b)
#     return y


In [111]:
# 가중치 W를 생성하는 시점을 늦추는 방식(초기화때가 아닌 forward때 생성)
class LLinear(Layer):
  def __init__(self, out_size, nobias=False, dtype=np.float32, in_size=None):
    super().__init__()
    self.in_size=in_size
    self.out_size=out_size
    self.dtype=dtype

    self.W=Parameter(data=None, name='W')
    if self.in_size is not None:
      self._init_W()

    if nobias:
      self.b=None
    else:
      self.b=Parameter(np.zeros(out_size, dtype=dtype),name='b')

  def _init_W(self):
    I,O=self.in_size, self.out_size
    W_data=np.random.randn(I,O).astype(self.dtype) * np.sqrt(1/I)
    self.W.data=W_data

  def forward(self, x):
    if self.W.data is None:
      self.in_size=x.shape[1]
      self._init_W()

    y=linear(x, self.W, self.b)
    return y


In [112]:
# Layer 이용한 신경망 구현
np.random.seed(0)
x=np.random.rand(100,1)
y=np.sin(2*np.pi*x)+np.random.rand(100,1)

l1=LLinear(10) # 출력 크기 지정함
l2=LLinear(1)

def predict(x):
  y=l1(x)
  y=sigmoid(y)
  y=l2(y)
  return y

lr=0.2
iters=10000

for i in range(iters):
  y_pred=predict(x)
  loss=mean_squared_error(y,y_pred)

  l1.cleargrads()
  l2.cleargrads()
  loss.backward()

  for l in [l1,l2]:
    for p in l.params():
      p.data-=lr*p.grad.data

  if i%1000==0:
    print(loss)

variable(0.8165178492839196)
variable(0.24990280802148895)
variable(0.24609876581126014)
variable(0.2372159081431807)
variable(0.20793216413350174)
variable(0.12311905720649353)
variable(0.07888166506355147)
variable(0.07655073683421637)
variable(0.0763780308623822)
variable(0.07618764131185572)


In [94]:
# Linear class를 개별적으로 다루는 부분이 어색함

## 계층을 모아두는 계층